In [1]:
import pickle
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time

Check https://www.datos.gov.co/resource/33dq-ab5a.json?id_persona_pd=0000000004

In [2]:
data=pickle.load(open("data/ID_PERSONA_PD.pb","rb"))

In [3]:
data= [str(x).zfill(10) for x in data]

In [4]:
len(data)

79679

In [5]:
data=sorted(list(set(data)))

In [6]:
len(data)

62784

In [7]:
RESTORE=False
if RESTORE:
    df=pd.read_json('cvlac_bak.json',dtype={'ID_PERSONA_PD':str})
    for start in range(len(data)):
        if data[start]==df.iloc[-1]['ID_PERSONA_PD']:
            break    
else:
    start=-1
    df=pd.DataFrame()

In [9]:
start

-1

Test

In [11]:
scienti='https://scienti.minciencias.gov.co/cvlac/visualizador/generarCurriculoCv.do?cod_rh='
#for cvlac in data[start+1:]:
if True:
    cvlac='0000893390'
    url=f'{scienti}{cvlac}'

    dd={'ID_PERSONA_PD':cvlac}

    r=requests.get(url,verify=False)

assert r.status_code==200

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'scienti.minciencias.gov.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [9]:
#cvlac='0000893390'
#cvlac='0000000000'
#cvlac='0000000004'

scienti='https://scienti.minciencias.gov.co/cvlac/visualizador/generarCurriculoCv.do?cod_rh='
for cvlac in data[start+1:]:
    url=f'{scienti}{cvlac}'

    dd={'ID_PERSONA_PD':cvlac}

    try:
        r=requests.get(url,verify=False)
    except:
        continue

    if not r.text:
        continue

    soup = BeautifulSoup(r.text, 'lxml') # Parse the HTML as a string
    tables = soup.find_all('table')

    #1: Full names
    if len(tables)>2:
        t=tables[1]
    else:
        dd['Nombre']=''
        continue

    tr=pd.read_html( t.decode() )[0].to_dict(orient='records')

    for d in tr:
        if d and isinstance( d.get(0),str) and isinstance( d.get(1),str):
            dd[d.get(0)]=d.get(1).replace('\xa0', ' ')
        else:
            continue

    #2. Academic Social Networks  and authors Ids
    t=tables[2]

    #2.a) Academic Social Networks (Google Scholar) 
    next=2
    try:
        ids=t.find_all('h3')[0].text
    except:
        ids=''
    if ids=='Redes sociales académicas':
        next=3
        l=t.find_all('a')
        for x in l:
            try:
                dd[x.text.split(' (')[0]]=x['href']
            except:
                continue
    #2.b) authors Ids (ORCID, Scopus, ...)
    try:
        t=tables[next]
    except:
        continue
                
    try:
        ids=t.find_all('h3')[0].text
    except:
        ids=''
    if ids=='Identificadores de autor':
        l=t.find_all('a')
        for x in l:
            try:
                dd[re.search('\(([\w]+)\)', x.text).groups()[0]]=x['href']
            except:
                continue
                
    #Store date
    df=df.append(dd,ignore_index=True)
    print(str( df.shape[0] ).zfill(5),end='\r')
    if df.shape[0]%100==0:
        df.to_json('cvlac.json',orient='records')
    time.sleep(0.3)
df.to_json('cvlac.json',orient='records')

In [ ]:
pd.DataFrame(lcvl)

API: 

Investigadores

In [1]:
import pandas as pd

In [9]:
cvlac='0000177733'
pd.read_json(f'https://www.datos.gov.co/resource/bqtm-4y2h.json?id_persona_pr={cvlac}')

,nme_convocatoria,ano_convo,nme_genero_pr,nme_pais_nac_pr,nme_region_nac_pr,nme_departamento_nac_pr,nme_municipio_nac_pr,cod_dane_nac_pr,id_niv_form_pr,nme_niv_form_pr,...,nme_region_gr,nme_departamento_gr,nme_municipio_gr,cod_dane_gr,id_area_con_gr,nme_area_esp_gr,nme_area_gr,nme_gran_area_gr,id_persona_pr,id_convocatoria
0,Convocatoria 640 de 2013,2013-01-01T00:00:00.000,Masculino,Colombia,Eje Cafetero,Antioquia,Fredonia,5282,5,Postdoctorado,...,Eje Cafetero,Antioquia,Medellín,5001,1C03,Física de partículas y campos,Ciencias físicas,Ciencias naturales,177733,16
1,Convocatoria 693 de 2014,2014-01-01T00:00:00.000,Masculino,Colombia,Eje Cafetero,Antioquia,Fredonia,5282,5,Postdoctorado,...,Eje Cafetero,Antioquia,Medellín,5001,1C03,Física de partículas y campos,Ciencias físicas,Ciencias naturales,177733,17
2,Convocatoria 737 de 2015,2015-01-01T00:00:00.000,Masculino,Colombia,Eje Cafetero,Antioquia,Fredonia,5282,5,Postdoctorado,...,Eje Cafetero,Antioquia,Medellín,5001,1C03,Física de partículas y campos,Ciencias físicas,Ciencias naturales,177733,18
3,Convocatoria 781 de 2017,2017-01-01T00:00:00.000,Masculino,Colombia,Eje Cafetero,Antioquia,Fredonia,5282,5,Postdoctorado,...,Eje Cafetero,Antioquia,Medellín,5001,1C,A - Ciencias físicas,Ciencias físicas,Ciencias naturales,177733,19
4,Convocatoria 833 de 2018,2019-12-06T00:00:00.000,Masculino,Colombia,Región Eje Cafetero,Antioquia,Fredonia,5282,5,Postdoctorado/Estancia postdoctoral,...,Región Eje Cafetero,Antioquia,Medellín,5001,1C03,Física de Partículas y Campos,Ciencias Físicas,Ciencias Naturales,177733,20


Productos

In [10]:
import pandas as pd

In [11]:
cvlac='0000177733'
#cvlac='0000119111'
pr=pd.read_json(f'https://www.datos.gov.co/resource/33dq-ab5a.json?id_persona_pd={cvlac}')

In [12]:
pr[:3]

,id_convocatoria,nme_convocatoria,ano_convo,id_producto_pd,nme_clase_pd,nme_tipo_medicion_pd,nme_tipologia_pd,id_tipo_pd_med,nme_categoria_pd,fcreacion_pd,nme_producto_pd,cod_grupo_gr,nme_grupo_gr,id_persona_pd
0,16,Convocatoria 640 de 2013,2013-01-01T00:00:00.000,ART-0000177733-9,Nuevo conocimiento,Nuevo conocimiento Top,Artículos de investigación,ART-ART_A1,Artículos de investigación Con Calidad A1,2013-05-01T00:00:00.000,Baryonic violation of R parity from anomalous ...,COL0008423,Grupo de Fenomenologia de Interacciones Fundam...,177733
1,16,Convocatoria 640 de 2013,2013-01-01T00:00:00.000,PID-0000177733-11,Formación de recurso humano,Formación de recursos humano Tipo B,Proyecto y Desarrollo,PID-PID_A,Proyecto de investigación y Desarrollo Con Cal...,2013-02-01T00:00:00.000,Implicaciones de modelos supersimétricos con v...,COL0008423,Grupo de Fenomenologia de Interacciones Fundam...,177733
2,16,Convocatoria 640 de 2013,2013-01-01T00:00:00.000,TP-0000177733-78,Formación de recurso humano,Formación de recursos humano Tipo B,Tesis de pregrado,TP-TP_A,Tesis de pregrado Con Calidad A,2010-01-01T00:00:00.000,Modelos de materia oscura,COL0008423,Grupo de Fenomenologia de Interacciones Fundam...,177733


In [13]:
pr['nme_tipologia_pd'].unique()

array(['Artículos de investigación', 'Proyecto y Desarrollo',
       'Tesis de pregrado', 'Evento científico', 'Tesis de doctorado',
       'Tesis de maestria', 'Artículos', 'Informe', 'Edicion',
       'Proyecto de investigacion y desarrollo',
       'Informe Final de Investigación'], dtype=object)

In [14]:
gr=pd.read_json('https://www.datos.gov.co/resource/hrhc-c4wu.json?cod_grupo_gr=COL0008423')

In [15]:
gr

,id_convocatoria,nme_convocatoria,ano_convo,cod_grupo_gr,nme_grupo_gr,fcreacion_gr,nme_municipio_gr,nme_departamento_gr,nme_pais_gr,nme_region_gr,cod_dane_gr,id_area_con_gr,nme_area_gr,nme_gran_area_gr,nme_clasificacion_gr,orden_clas_gr,edad_anos_gr,inst_aval
0,16,Convocatoria 640 de 2013,2013-01-01T00:00:00.000,COL0008423,Grupo de Fenomenologia de Interacciones Fundam...,1979-01-12T00:00:00.000,Medellín,Antioquia,Colombia,Eje Cafetero,5001,1C,Ciencias físicas,Ciencias naturales,A1,5,34.17,Universidad De Antioquia - Udea
1,17,Convocatoria 693 de 2014,2014-01-01T00:00:00.000,COL0008423,Grupo de Fenomenologia de Interacciones Fundam...,1979-01-12T00:00:00.000,Medellín,Antioquia,Colombia,Eje Cafetero,5001,1C,Ciencias físicas,Ciencias naturales,A1,5,34.67,Universidad De Antioquia - Udea
2,18,Convocatoria 737 de 2015,2015-01-01T00:00:00.000,COL0008423,Grupo de Fenomenologia de Interacciones Fundam...,1979-01-12T00:00:00.000,Medellín,Antioquia,Colombia,Eje Cafetero,5001,1C,Ciencias físicas,Ciencias naturales,A1,5,35.67,Universidad De Antioquia - Udea
3,19,Convocatoria 781 de 2017,2017-01-01T00:00:00.000,COL0008423,Grupo de Fenomenologia de Interacciones Fundam...,1979-01-12T00:00:00.000,Medellín,Antioquia,Colombia,Eje Cafetero,5001,1C,Ciencias físicas,Ciencias naturales,A1,5,38.00,Universidad De Antioquia - Udea
4,20,Convocatoria 833 de 2018,2019-06-12T00:00:00.000,COL0008423,Grupo de Fenomenologia de Interacciones Fundam...,1979-01-12T00:00:00.000,Medellín,Antioquia,Colombia,Eje Cafetero,5001,1C,Ciencias Físicas,Ciencias Naturales,A1,4,39.00,Universidad De Antioquia - Sede Carmen De Viboral


In [2]:
import pandas as pd

In [3]:
df=pd.read_csv('4-cvlac.csv')

In [ ]:
df[df['Nombre'].str.title()=='Artevic Holgueras Galán']

In [ ]:
df['Nombre'].str.split().apply(len)

In [18]:
df[(df['Nacionalidad'].apply(lambda s: s in ['Extranjero - otra','Brasilera']) ) & (df['Nombre'].str.split().apply(len)==3 )].shape

(657, 18)

In [8]:
df[df['Nacionalidad']==].shape

(130, 18)